In [ ]:
!pip install contextily
!pip install geopandas
!pip install folium matplotlib mapclassify

# New Section

In [ ]:
from typing import Optional
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

# from vizLib import read_data, merge_dataframes, generate_population_map
# from vizLib import *
from TermProject.vizLib.vizLib import *
from TermProject.vizLib.utils import *

In [ ]:
gdf, demo_data = read_data('TermProject/lda_000b16a_e.shp', 'TermProject/project_data.csv', 'TermProject/project_header.txt', ' - ', 0)

population = demo_data[['GEO UID', 'Population, 2016','Total-Private households','Population density per square kilometre','Total_percent_pop_65', 'Total_percent_pop_0_14', 'Total_percent_pop_15_64', 'Average age of the population']].iloc[1:].copy()
population.rename(columns={'GEO UID': 'DAUID'}, inplace= True)

languages = demo_data[['GEO UID', 'Population, 2016','Total_knowledge_languages','Total_knowledge_eng','Total_knowledge_fre', 'Total_knowledge_fre_eng', 'Total_knowledge_none', 'Mother Tongue_Total', 'Mother Tongue_non_official_lang','Mother Tongue_official_lang']].iloc[1:].copy()
languages.rename(columns={'GEO UID': 'DAUID', 'Total_knowledge_none': 'Total_knowledge_none', 'Mother Tongue_Total': 'Mother_tongue_Total', 'Mother Tongue_non_official_lang': 'Mother_tongue_non_official_lang','Mother Tongue_official_lang': 'Mother_tongue_official_lang'}, inplace=True)

diversity = demo_data[['GEO UID', 'birth_total', 'birth_Americas', 'birth_Europe', 'birth_Africa',
       'birth_Asia', 'birth_Oceania_and_other']].iloc[1:].copy()
diversity.rename(columns={'GEO UID': 'DAUID', 'Population, 2016': 'population'}, inplace= True)

population['age dependency ratio'] = (population['Total_percent_pop_0_14'] + population['Total_percent_pop_65'])/population['Total_percent_pop_15_64']

gdf['DAUID'] = gdf['DAUID'].astype('int')
merged_population = merge_dataframes(df1=gdf, df2=population, common_column='DAUID',filter_column1= 'CDNAME', filter_value1= 'Montréal')
merged_languages = merge_dataframes(df1=gdf, df2=languages, common_column='DAUID',filter_column1= 'CDNAME', filter_value1= 'Montréal')
merged_diversity = merge_dataframes(df1=gdf, df2=diversity, common_column='DAUID',filter_column1= 'CDNAME', filter_value1= 'Montréal')



In [ ]:
columns = {
    'Average age of the population': 'Average age of the population',
    'Total_percent_pop_65': 'Total_percent_pop_65',
    'Total_percent_pop_15_64': 'Total_percent_pop_15_64',
    'age dependency ratio': 'age dependency ratio'
}

generate_population_choropleth(merged_population, columns)

In [ ]:

# specify dictionary mapping column names to the required keys
col_dict = {
    "Total_knowledge_eng": "Total_knowledge_eng",
    "Total_knowledge_fre": "Total_knowledge_fre",
    "Total_knowledge_fre_eng": "Total_knowledge_fre_eng",
    "Total_knowledge_none": "Total_knowledge_none",
    "Total_knowledge_languages": "Total_knowledge_languages",
    "Mother_tongue_official_lang": "Mother_tongue_official_lang",
    "Mother_tongue_non_official_lang": "Mother_tongue_non_official_lang",
    "Mother_tongue_Total": "Mother_tongue_Total",
}

# call function with dataframe and column dictionary
merged_languages = calculate_language_ratios(merged_languages, col_dict)


columns = {
    'Population who know English': 'ratio_eng',
    'Population who know French': 'ratio_fre',
    'Population who know both English and French': 'ratio_both',
    'Population who know neither English nor French': 'ratio_none'
}
plot_language_choropleth(merged_languages, columns)

In [ ]:
cols_to_plot = {
    'Ratio - Population with Official Language as mother tongue': 'ratio_mother_off',
    'Ratio - Population with Un-official Language as mother tongue': 'ratio_mother_unoff'
}

plot_mother_tongue_ratio(merged_languages, cols_to_plot)


In [ ]:
display(generate_population_map(merged_population))

In [ ]:
education = demo_data[['GEO UID', 'Population, 2016','total_education', 'total_no_certificate', 'total_secondary','total_post_secondary', 'total_education_24_65','total_no_certificate_24_65', 'total_secondary_24_65','total_post_secondar_24_65']].iloc[1:].copy()
education.rename(columns={'GEO UID': 'DAUID', 'Population, 2016': 'population'}, inplace= True)


highest_ratio, dauid_with_highest_ratio, education_level_with_highest_ratio = get_highest_education_ratio(
    education, 'population', 'total_post_secondar_24_65')
print('Highest ratio:', highest_ratio)
print('DAUID with highest ratio:', dauid_with_highest_ratio)
print('Education level with highest ratio:', education_level_with_highest_ratio)


merged_education = merge_dataframes(df1=gdf, df2=education, common_column='DAUID',filter_column1= 'CDNAME', filter_value1= 'Montréal')

map = create_education_levels_map(merged_education, 'total_no_certificate', 'total_secondary', 'total_post_secondary', 'population')
display(map)

In [ ]:
display(plot_birth_density_maps(merged_diversity))